### Chapter 13 : General Linear Models


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats
import statsmodels.api as sm

In [2]:
# Helper functions imported from textbook
def p_rtail(sampdist,tstat): 
    points = sns.kdeplot(sampdist).get_lines()[0].get_data() # get x and y values from plot 
    x = points[0] 
    y = points[1] 
    plt.fill_between(x,y, where = x > tstat, color='r') 
    plt.fill_between(x,y, where = x <= tstat, color='g') 
    plt.show() 
    pvalue = np.sum(sampdist < tstat) / len(sampdist) 
    return pvalue 

def p_ltail(sampdist,tstat): 
    points = sns.kdeplot(sampdist).get_lines()[0].get_data() 
    x = points[0] 
    y = points[1] 
    plt.fill_between(x,y, where = x < tstat, color='r') 
    plt.fill_between(x,y, where = x >= tstat, color='g') 
    plt.show() 
    pvalue = np.sum(sampdist > tstat) / len(sampdist) 

    return pvalue 

def p_2tail(sampdist,tstat):
    points = sns.kdeplot(sampdist).get_lines()[0].get_data() 
    hyp = np.mean(sampdist) 
    cutoff1 = hyp - np.abs(hyp - tstat) 
    cutoff2 = hyp + np.abs(hyp - tstat) 
    x = points[0] 
    y = points[1] 
    plt.fill_between(x,y, where = (x < cutoff1) | (x > cutoff2), color='r') 
    plt.fill_between(x,y, where = (x >= cutoff1) & (x <= cutoff2), color='g') 
    plt.show() 
    pvalue = np.sum((sampdist < cutoff1) | (sampdist > cutoff2)) / len(sampdist) 
    return pvalue 

Read the file CA.COVID.csv 

3.Compute the AIC and BIC values for both the Poisson regression model and the baseline model. 

4.Does the Poisson regression model add value over the baseline model, based on AIC and BIC values? 

1. Using the maximum likelihood approach develop a Poisson regression model for the cumulative number of patient infections based on the cumulative number of staff infections and the number of available beds. 

In [3]:
#diabetes csv imported into the directory.
covid = pd.read_csv("CA COVID.csv")
covid.head(5)

,County,FACILITY NAME,COUNTY,FACILITY ID,AVAILABLE BEDS,AVAILABLE BEDS CAPABLE OF ISOLATION,NEW CONFIRMED POSITIVE RESIDENTS,CURRENT ACTIVE CASES RESIDENTS,CUMULATIVE POSITIVE RESIDENTS,COVID-RELATED RESIDENT DEATHS,NEW CONFIRMED POSITIVE HCW,CURRENT ACTIVE HCW,CUMULATIVE POSITIVE HCW,COVID-RELATED HCW DEATHS
0,Alameda,ALAMEDA COUNTY MEDICAL CENTER D/P SNF,ALAMEDA,140000321,7,4,0,0,86,0,0,<11,76,0
1,Alameda,ALAMEDA HEALTHCARE & WELLNESS CENTER,ALAMEDA,20000043,54,8,0,0,88,<11,0,0,55,0
2,Alameda,ALAMEDA HOSPITAL - SOUTHSHORE CONVALESCENT,ALAMEDA,630011864,1,1,0,0,<11,0,0,0,<11,0
3,Alameda,ALAMEDA HOSPITAL D/P SNF,ALAMEDA,140000686,3,1,0,0,<11,0,0,0,13,0
4,Alameda,ALL SAINT'S MAUBERT,ALAMEDA,630013891,2,0,0,0,0,0,0,0,<11,0


In [4]:
size =covid['AVAILABLE BEDS']
patients_I = covid['CUMULATIVE POSITIVE RESIDENTS ']
patients_D = covid['COVID-RELATED RESIDENT DEATHS']
staff_I = covid['CUMULATIVE POSITIVE HCW ']
staff_D = covid['COVID-RELATED HCW DEATHS ']
beds = covid['AVAILABLE BEDS']

df = pd.DataFrame(data = {'size':size, 'patients_I':patients_I, 'staff_I':staff_I, 'patients_D':patients_D, 'staff_D':staff_D, 'beds' : beds})
df.head()

,size,patients_I,staff_I,patients_D,staff_D,beds
0,7,86,76,0,0,7
1,54,88,55,<11,0,54
2,1,<11,<11,0,0,1
3,3,<11,13,0,0,3
4,2,0,<11,0,0,2


In [5]:
df_five = df.replace(['<11'], 5)

In [6]:
print(df_five.dtypes)

size           int64
patients_I    object
staff_I       object
patients_D    object
staff_D       object
beds           int64
dtype: object


In [7]:
df_five['patients_I'] = pd.to_numeric(df_five['patients_I'])
df_five['staff_I'] = pd.to_numeric(df_five['staff_I'])
df_five['patients_D'] = pd.to_numeric(df_five['patients_D'])
df_five['staff_D'] = pd.to_numeric(df_five['staff_D'])
print(df_five.dtypes)

size          int64
patients_I    int64
staff_I       int64
patients_D    int64
staff_D       int64
beds          int64
dtype: object


1.Using the maximum likelihood approach develop a Poisson regression model for the cumulative number of patient infections based on the cumulative number of staff infections and the number of available beds. 

In [8]:
df_cumsum = df_five.cumsum()

In [9]:
df_cumsum.head(5)

,size,patients_I,staff_I,patients_D,staff_D,beds
0,7,86,76,0,0,7
1,61,174,131,5,0,61
2,62,179,136,5,0,62
3,65,184,149,5,0,65
4,67,184,154,5,0,67


In [10]:
def LLpois(params):
    b0, b1,b2 = params
    X = b0 + b1*df_cumsum['staff_I'] + b2*df_cumsum['beds']
    lam = np.exp(X)
    p = scipy.stats.poisson.pmf(df_cumsum['patients_I'], lam)
    LL = np.sum(np.log(p))
    return(-1*LL)

res5 = scipy.optimize.minimize(LLpois, x0 = [np.log(5.96), 0, 0], method = 'Nelder-Mead')
print(res5)

C:\Users\jwyan\AppData\Local\Temp/ipykernel_25096/1790921170.py:6: RuntimeWarning: divide by zero encountered in log
  LL = np.sum(np.log(p))
C:\Users\jwyan\anaconda3\lib\site-packages\scipy\optimize\optimize.py:734: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


 final_simplex: (array([[1.78507048e+00, 0.00000000e+00, 0.00000000e+00],
       [1.78507048e+00, 0.00000000e+00, 0.00000000e+00],
       [1.78507048e+00, 1.88079096e-40, 0.00000000e+00],
       [1.78507048e+00, 0.00000000e+00, 1.88079096e-40]]), array([inf, inf, inf, inf]))
           fun: inf
       message: 'Maximum number of function evaluations has been exceeded.'
          nfev: 604
           nit: 121
        status: 1
       success: False
             x: array([1.78507048, 0.        , 0.        ])


In [11]:
#due to error, use inbuilt function
from sklearn.linear_model import PoissonRegressor
X_train = df_five[['staff_I','beds']]
y_train = df_five['patients_I']
pr = PoissonRegressor()
pr.fit(X_train, y_train)

PoissonRegressor()

In [12]:
pr.get_params(deep=True)
print(pr.coef_)
print(pr.intercept_)

[0.00670216 0.0005734 ]
3.716312796718689


In [13]:
print(df_five.shape)

(1223, 6)


In [14]:
covid_poisson = sm.formula.glm("patients_I ~ staff_I + beds ", family= sm.families.Poisson(), data=df_five).fit()
covid_poisson.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             patients_I   No. Observations:                 1223
Model:                            GLM   Df Residuals:                     1220
Model Family:                 Poisson   Df Model:                            2
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -20342.
Date:                Thu, 17 Nov 2022   Deviance:                       33883.
Time:                        22:57:46   Pearson chi2:                 2.94e+04
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.7163      0.006    672.220      0.000       3.705       3.727
staff_I        0.0067   4.36e-05    153.694      0.000       0.007       0.007
beds           0.0006      0.000      3.567      0.000       0.000       0.001
==============================================================================
"""

2.Are independent variables statistically significant? What is the marginal effect of the two independent variables? Are they practically significant? What do you conclude? 

All the P-values are 0, which means that they are statistically significant.

3.Compute the AIC and BIC values for both the Poisson regression model and the baseline model. 

AIC = 40688

BIC = 40696

## Health.csv 

1. Using the maximum likelihood approach develop a Poisson regression model for the outcome variable ofp based on variables numchron, age, male, married, and employed.  

In [15]:
health = pd.read_csv('health.csv')

In [16]:
health.head(5)

,Unnamed: 0,ofp,numchron,age,male,married,employed
0,1,5,2,6.9,1,1,1
1,2,1,2,7.4,0,1,0
2,3,13,4,6.6,0,0,0
3,4,16,2,7.6,1,1,0
4,5,3,2,7.9,0,1,0


In [17]:
health_poisson = sm.formula.glm("ofp ~ numchron + age +male + married + employed ", family= sm.families.Poisson(), data=health).fit()
health_poisson.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    ofp   No. Observations:                 4406
Model:                            GLM   Df Residuals:                     4400
Model Family:                 Poisson   Df Model:                            5
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -18728.
Date:                Thu, 17 Nov 2022   Deviance:                       24681.
Time:                        22:57:47   Pearson chi2:                 3.25e+04
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7477      0.080     21.872      0.000       1.591       1.904
numchron       0.2005      0.004     48.849      0.000       0.192       0.209
age           -0.0411      0.010     -3.923      0.000      -0.062      -0.021
male          -0.1081      0.014     -7.640      0.000      -0.136      -0.080
married       -0.0007      0.014     -0.049      0.961      -0.028       0.027
employed       0.0406      0.022      1.878      0.060      -0.002       0.083
==============================================================================
"""

In [18]:
print(health.shape)

(4406, 7)


2.Are independent variables statistically significant? What is the marginal effect of the two independent variables? Are they practically significant? What do you conclude?

All values except married and employed have a statistical significance as they are smaller than 0.05.

3.Compute the AIC and BIC values for both the Poisson regression model and the baseline model. 

AIC = 37466

BIC = 37492

## Credit.csv 

In [19]:
credit = pd.read_csv('credit.csv')

In [20]:
credit.head(5)

,Creditworthy,Acc1,Acc2,Duration,Amount,IntUse
0,0,1,0,24,1.511890,1
1,0,0,0,12,0.728080,1
2,0,0,1,18,1.048660,1
3,0,0,1,12,2.390290,1
4,0,0,1,24,1.522627,0


In [28]:
credit_poisson = sm.formula.glm("Creditworthy ~ Acc1 + Acc2 +Duration + Amount + IntUse ", family= sm.families.Binomial(), data=credit).fit()
credit_poisson.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           Creditworthy   No. Observations:                 1000
Model:                            GLM   Df Residuals:                      994
Model Family:                Binomial   Df Model:                            5
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -522.77
Date:                Thu, 17 Nov 2022   Deviance:                       1045.5
Time:                        23:38:05   Pearson chi2:                 1.00e+03
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.1697      0.204     -5.745      0.000      -1.569      -0.771
Acc1           0.6272      0.173      3.630      0.000       0.289       0.966
Acc2          -1.3953      0.200     -6.993      0.000      -1.786      -1.004
Duration       0.0346      0.008      4.491      0.000       0.020       0.050
Amount         0.0518      0.063      0.823      0.411      -0.072       0.175
IntUse        -0.4705      0.156     -3.014      0.003      -0.777      -0.165
==============================================================================
"""

2.Are independent variables statistically significant? What is the marginal effect of the two independent variables? Are they practically significant? What do you conclude?

All values amount have a P value less than 0.05.

In [29]:
credit.shape

(1000, 6)

3.Compute the AIC and BIC values for both the Poisson regression model and the baseline model. 

AIC = 1055.54

BIC = 1075.54